In [9]:
import csv
import requests
import re
import numpy as np
import pandas as pd


In [ ]:

URL1 ="https://pubmed.ncbi.nlm.nih.gov/?term=(cerebral%20palsy%20or%20CP)%20and%20(repetitive%20transcranial%20magnetic%20stimulation%20or%20rTMS%20or%20tDCS%20or%20transcranial%20direct%20current%20stimulation)%20and%20(motor)&sort=pubdate&size=200"
URL2 ="https://pubmed.ncbi.nlm.nih.gov/?term=(cerebral%20palsy%20or%20CP)%20and%20(repetitive%20transcranial%20magnetic%20stimulation%20or%20rTMS%20or%20tDCS%20or%20transcranial%20direct%20current%20stimulation)%20and%20(motor)&sort=pubdate&size=200&page=2"
selected = 'page1'

if selected =='page1':
    res = requests.get(URL1) # 
elif selected =='page2':
    res = requests.get(URL2) # 

print("requested url:",res.url) # 접속한 url 정보

from bs4 import BeautifulSoup
bsObj = BeautifulSoup(res.text, "html.parser")

# 첫번째 a 태그 뽑아오기
list_title = bsObj.find_all("a","docsum-title")
data = pd.DataFrame(columns=['title'])
titles = []
for title in list_title:
    # 논문 이름 뽑기
    raw  = title.text
    processed = re.sub("\s +","", raw).strip()
    titles.append(processed)
    #
list_paper_url_info = bsObj.find_all("button", {"class":"share-search-result trigger result-action-trigger share-dialog-trigger"})
for paper_url_info in list_paper_url_info[::2]:
    paper_url= paper_url_info["data-permalink-url"] #이렇게 하면 파일마다 주소로 접근할 수 있음
    res_paper = requests.get(paper_url) # 접속한 url 정보
    bsObj_paper = BeautifulSoup(res_paper.text, "html.parser")

data['title'] = titles

data.to_csv(f'title_{selected}.csv')
# 참고: https://swlock.blogspot.com/2019/08/web-page-crawling-scraping-python.html
# # 글자만 뽑아오기
# print(first_a.text)
# print("href",first_a.get("href"))

# # 이 주소로 접속 하면 안됨
# print(URL+first_a.get("href"))

In [10]:
import openpyxl


openpyxl

<module 'openpyxl' from 'c:\\Users\\orien\\anaconda3\\envs\\buIMU\\lib\\site-packages\\openpyxl\\__init__.py'>

In [11]:
olddata = pd.read_csv('./title_page1.csv', usecols=['title'])

newdata = pd.read_excel('./Individual studies_revised.xlsx',  usecols=['Title'])



In [12]:
olddata['ver'] = 'old'
newdata['ver'] = 'new'
print(newdata)
print(olddata)

                                                 Title  ver
0    Do Adults with Stroke have Altered Interhemisp...  new
1    Non-invasive Brain Stimulation for Chronic Pai...  new
2    Physical therapy in children with cerebral pal...  new
3    Exploring Clinical and Neurophysiological Fact...  new
4    Robotic mapping of motor cortex in children wi...  new
..                                                 ...  ...
223  Transcranial magnetic stimulation as a diagnos...  new
224  Transcranial magnetic coil stimulation of moto...  new
225  Altered corticospinal projections to lower lim...  new
226  Estimation of facial central motor delay by el...  new
227  [Assessment of motor cranial nerve functions u...  new

[228 rows x 2 columns]
                                                 title  ver
0    Do Adults with Stroke have Altered Interhemisp...  old
1    Repetitive Peripheral Magnetic Stimulation to ...  old
2    Non-invasive Brain Stimulation for Chronic Pai...  old
3    Physical th

In [ ]:
A =A.strip("'-.")
B =B.strip("'-.")
C =C.rstrip("'-.")

In [27]:
id_dropped = set(olddata['title'].strip("'-.")) - set(newdata['Title'].strip("'-."))
id_added = set(newdata['Title'].strip("'-.")) - set(olddata['title'].strip("'-."))

# print('삭제된 아이템: ',id_dropped)
# print('추가된 아이템: ',id_added)
set(olddata['title'])

AttributeError: 'Series' object has no attribute 'strip'

In [5]:
A = set([1,2,3,4,56,7,2,2,3,2,1])
B = set([1,2,3])
A - B, B-A

({4, 7, 56}, set())

In [88]:
df_merge = pd.DataFrame()

df_merge['New'] = newdata['Title']
df_merge['Old'] = olddata['title']

df_merge.to_excel('result.xlsx')


In [106]:
df1 = pd.DataFrame(df_merge['New'])
print(df1)

                                                   New
0    Do Adults with Stroke have Altered Interhemisp...
1    Non-invasive Brain Stimulation for Chronic Pai...
2    Physical therapy in children with cerebral pal...
3    Exploring Clinical and Neurophysiological Fact...
4    Robotic mapping of motor cortex in children wi...
..                                                 ...
223  Transcranial magnetic stimulation as a diagnos...
224  Transcranial magnetic coil stimulation of moto...
225  Altered corticospinal projections to lower lim...
226  Estimation of facial central motor delay by el...
227  [Assessment of motor cranial nerve functions u...

[228 rows x 1 columns]


In [26]:
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

A = "''''''인공지능 학습을 통한 IMU 센서의 활용가능성 탐구에 대하여-"

C = "리뷰-인공지능 학습을 통한 imu 센서의 활용가능성 탐구에 대하여"
A =A.strip("'-.")
B =B.strip("'-.")
C =C.rstrip("'-.")
similar(A,B),similar(B,C)

(1.0, 0.9565217391304348)

'인공지능 학습을 통한 IMU 센서의 활용가능성 탐구에 대하여-'

In [107]:
df2 = pd.DataFrame(df_merge['Old'])
print(df2)

                                                   Old
0    Do Adults with Stroke have Altered Interhemisp...
1    Repetitive Peripheral Magnetic Stimulation to ...
2    Non-invasive Brain Stimulation for Chronic Pai...
3    Physical therapy in children with cerebral pal...
4    Exploring Clinical and Neurophysiological Fact...
..                                                 ...
223                                                NaN
224                                                NaN
225                                                NaN
226                                                NaN
227                                                NaN

[228 rows x 1 columns]


In [111]:
##완벽히 같은지는 볼 수 있음
df1.reset_index(drop=True).equals(df2.reset_index(drop=True))

False

In [133]:
print(df1['New'].isin(df2['Old']))     

0      False
1      False
2      False
3      False
4      False
       ...  
223    False
224    False
225    False
226    False
227    False
Name: New, Length: 228, dtype: bool


In [130]:
from numpy import NaN


a = df2[~df2.Old.isin(df1.New)]
for nothing in a:
    if a == NaN:
        print('X')


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [127]:
df_merge['difference'] = a
df_merge.head()
df_merge.to_excel('result2.xlsx')

In [122]:
print(df2['Old'].isin(df1)==False)

0      True
1      True
2      True
3      True
4      True
       ... 
223    True
224    True
225    True
226    True
227    True
Name: Old, Length: 228, dtype: bool


In [117]:
df_diff = df1 - df2
print(df_diff)

     New  Old
0    NaN  NaN
1    NaN  NaN
2    NaN  NaN
3    NaN  NaN
4    NaN  NaN
..   ...  ...
223  NaN  NaN
224  NaN  NaN
225  NaN  NaN
226  NaN  NaN
227  NaN  NaN

[228 rows x 2 columns]


In [86]:
df_dropped = olddata[olddata['title'].isin(id_dropped)].iloc[:,:-1]
df_added = newdata[newdata['Title'].isin(id_added)].iloc[:,:-1]

In [87]:

print(df_dropped)
print(df_added)

                                                 title
0    Do Adults with Stroke have Altered Interhemisp...
1    Repetitive Peripheral Magnetic Stimulation to ...
2    Non-invasive Brain Stimulation for Chronic Pai...
3    Physical therapy in children with cerebral pal...
4    Exploring Clinical and Neurophysiological Fact...
..                                                 ...
195    Corticospinal disinhibition during dual action.
196  Short-term effects of functional electrical st...
197  Electrophysiological assessment of the effect ...
198       Eye movement control by the cerebral cortex.
199  Electrical stimulation of the human common per...

[194 rows x 1 columns]
                                                 Title
0    Do Adults with Stroke have Altered Interhemisp...
1    Non-invasive Brain Stimulation for Chronic Pai...
2    Physical therapy in children with cerebral pal...
3    Exploring Clinical and Neurophysiological Fact...
4    Robotic mapping of motor cortex in c

In [ ]:
# 두 데이터프레임을 하나로 합칩니다.
df_concatted = pd.concat([olddata, newdata], ignore_index=True)
# 모든 컬럼의 내용이 중복되는 데이터는 삭제합니다.
changes = df_concatted.drop_duplicates(df_concatted.columns[:-1], keep='last')

# 남은 데이터 중 동일한 아이디 값이 두개 이상 존재한다면
# 정보가 변경된 데이터입니다.
duplicated_list = changes[changes['title'].duplicated()]['title'].to_list()
df_changed = changes[changes['title'].isin(duplicated_list)]
print(df_changed)

In [ ]:
df_changed_old = df_changed[df_changed['ver'] == 'old'].iloc[:,:-1]
df_changed_old.sort_values(by='title', inplace=True)

df_changed_new = df_changed[df_changed['ver'] == 'new'].iloc[:,:-1]
df_changed_new.sort_values(by='Title', inplace=True)


In [ ]:
df_info_changed = df_changed_old.copy()
for i in range(len(df_changed_new.index)):
    for j in range(len(df_changed_new.columns)):
        if (df_changed_new.iloc[i, j] != df_changed_old.iloc[i, j]):
            df_info_changed.iloc[i,j] = str(df_changed_old.iloc[i, j]) + " ==> " + str(df_changed_new.iloc[i,j])
            

# 오늘 배운 것

In [ ]:
paper_url = bsObj.find_all("button","data-permalink-url")

In [ ]:
# 개별 페이지 접속
PAPERURL ='https://pubmed.ncbi.nlm.nih.gov/8969779/'
res = requests.get(PAPERURL)
print("requested url:",res.url) # 접속한 url 정보
bsObj = BeautifulSoup(res.text, "html.parser")
# 첫번째 a 태그 뽑아오기
abstract = bsObj.find_all("div","abstract")

In [ ]:

for i in abstract:
    print(i.text)

In [ ]:
title

In [ ]:
bsObj